In [2]:
import argparse
import json
import logging
import shutil

import openrouteservice as ors
import pandas as pd
import numpy as np

from scripts.utils import load_config
from scripts.filepaths import FilePaths

/home/dabanto/miniconda3/envs/heal_project/lib/python3.13/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


ModuleNotFoundError: No module named 'scripts'

In [3]:
df = pd.read_csv("../run_pairs.csv")

df['id'] = np.arange(0, len(df))

del df['Unnamed: 0']

df

,lon,lat,lon2,lat2,distance,id
0,8.681673,49.436255,8.664183,49.381690,6.194010,0
1,8.626623,49.429120,8.638298,49.377357,5.813829,1
2,8.657504,49.418083,8.752665,49.396174,7.298887,2
3,8.747789,49.408750,8.605112,49.424165,10.455903,3
4,8.688200,49.369890,8.745959,49.424481,7.365509,4
...,...,...,...,...,...,...
85,8.637173,49.410095,8.690562,49.417784,3.953267,85
86,8.703606,49.366836,8.702268,49.416198,5.486261,86
87,8.714224,49.411343,8.641492,49.406389,5.287611,87
88,8.698501,49.403413,8.755992,49.424489,4.770830,88


In [4]:
df['lon'][2]

np.float64(8.657503792414388)

In [33]:
headers = {
        "headers": {
            "Accept": "application/json, application/geo+json, "
            "application/gpx+xml, img/png; charset=utf-8",
            "Content-Type": "application/json; charset=utf-8",
        }
    }

ors_url = "https://heal.openrouteservice.org/api-iso/ors/"

client = ors.Client(base_url=ors_url)

start_coordinate = [df['lon'][2], df['lat'][1]]
destination_coordinate = [df['lon2'][2], df['lat2'][1]]




parameters = {
        "coordinates": [start_coordinate, destination_coordinate],
        "instructions": "false",
        "preference": "recommended",
        "extra_info": ["csv"],
        "elevation": "true",
        "continue_straight": "true",
        "options": {
            "avoid_features": ["ferries"],
            "profile_params": {"weightings": {"csv_factor": 1, "csv_column": "noon"}},
        },
    }

ors_response = client.request(
        url=f"v2/directions/{profile}/geojson",
        post_json=parameters,
        requests_kwargs=headers,
    )

print(parameters)

with open("../data/run_v1/01_raw/01_routes/route_1.geojson", "w") as f:
    json.dump(ors_response, f)

{'coordinates': [[np.float64(8.657503792414388), np.float64(49.42912049732929)], [np.float64(8.752665401673024), np.float64(49.37735718455988)]], 'instructions': 'false', 'preference': 'recommended', 'extra_info': ['csv'], 'elevation': 'true', 'continue_straight': 'true', 'options': {'avoid_features': ['ferries'], 'profile_params': {'weightings': {'csv_factor': 1, 'csv_column': 'noon'}}}}


In [38]:
from tqdm.notebook import tqdm
from openrouteservice import client
import geopandas as gpd
import time
import json

err_iso = []

ors_url = "https://heal.openrouteservice.org/api-iso/ors/"
client = ors.Client(base_url=ors_url)

count_requests = 0

for _, row in tqdm(df.iterrows(), total=len(df)):

    try:
        start_coordinate = [row.lon, row.lat]
        destination_coordinate = [row.lon2, row.lat2]
        parameters['coordinates'] = [start_coordinate, destination_coordinate]
        parameters["options"]["profile_params"]["weightings"]["csv_column"] = "morning"
        profile = "foot-walking"

        ors_response = client.request(
            url=f"v2/directions/{profile}/geojson",
            post_json=parameters,
            requests_kwargs=headers,
        )

        with open(f"../data/run_v1/01_raw/01_routes/route_{row.id}.geojson", "w") as f:
            json.dump(ors_response, f)

        count_requests += 1

        if count_requests >= 10:
            print("Reached 10 iterations.")
            break
        else:
            time.sleep(0.3)

    except Exception as e:
        print(e)
        err_iso.append(row)




  0%|          | 0/90 [00:00<?, ?it/s]

Reached 10 iterations.


In [16]:
from itertools import product

list_a = ['k', 'l', 'm']

all_pairs = list(product(list_a, df.iterrows()))
print(len(all_pairs))


270


In [21]:
n = 2  # your input number

values = range(n)  # generates 0, 1, 2, ..., n-1 automatically
all_pairs = list(product(list_a, values))

In [22]:
print(len(all_pairs))

6
